In [13]:
from tools import model
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [39]:
data=pd.read_parquet("fig5_24_at25_bad_intsteps.parquet")
m=model(formula="simple_regret~algorithm+prior+(1|benchmark)",data=data)

In [40]:
# print(m.fixef)
print(m.ranef)
print(m.summary())

              X.Intercept.
JAHS-C10          2.709106
JAHS-CH           2.124731
JAHS-FM           0.832468
LC-126026        -0.697240
LC-167190        -0.651381
LC-168330        -0.568522
LC-168910        -0.604845
LC-189906        -0.647080
PD1-Cifar100     -0.572733
PD1-ImageNet     -0.629623
PD1-LM1B         -0.675560
PD1-WMT          -0.619322
Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: simple_regret~algorithm+prior+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 86400	 Groups: {'benchmark': 12.0}

Log-likelihood: -244152.467 	 AIC: 488316.933

Random effects:

                  Name     Var    Std
benchmark  (Intercept)   1.346  1.160
Residual                16.658  4.081

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE         DF  T-stat  P-val  Sig
(Intercept)     0.740   0.081    1.398  0.336     12.124   2.201  0.048    *
algorithm1     -0.393  -0.460   -0.327  0.034 

In [41]:
print(m.post_hoc("algorithm"))

P-values adjusted by tukey method for family of 3 estimates
(  algorithm  Estimate  2.5_ci  97.5_ci     SE      DF
1        PB     0.951   0.220    1.682  0.336  12.082
2        RS     0.558  -0.173    1.289  0.336  12.082
3        HB     0.658  -0.073    1.389  0.336  12.082,   Contrast  Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  P-val  Sig
1  PB - RS     0.393   0.314    0.473  0.034  86388.0  11.566  0.000  ***
2  PB - HB     0.293   0.214    0.373  0.034  86388.0   8.625  0.000  ***
3  RS - HB    -0.100  -0.180   -0.020  0.034  86388.0  -2.941  0.009   **)


In [47]:
md=smf.mixedlm("simple_regret~C(algorithm)+C(prior)",data,groups=data["benchmark"]).fit()
print(md.summary())

            Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: simple_regret
No. Observations:   86400   Method:             REML         
No. Groups:         12      Scale:              16.6590      
Min. group size:    7200    Log-Likelihood:     -244160.3521 
Max. group size:    7200    Converged:          Yes          
Mean group size:    7200.0                                   
-------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.446    0.353  1.265 0.206 -0.245  1.138
C(algorithm)[T.PB]  0.293    0.034  8.624 0.000  0.227  0.360
C(algorithm)[T.RS] -0.100    0.034 -2.941 0.003 -0.167 -0.033
C(prior)[T.bad]     0.423    0.028 15.244 0.000  0.369  0.478
Group Var           1.483    0.157                           



In [48]:
# Step 1: For all factors, create an array containing lists of all unique values of each factor
factors=["algorithm","prior"]
unique_factors=[data[f].unique() for f in factors]
combi_length = 1
for sublist in unique_factors:
    combi_length *= len(sublist)


grid=np.array(np.meshgrid(*unique_factors)).reshape(len(factors),combi_length).T
grid=pd.DataFrame(grid, columns = factors)
print(grid)

  algorithm prior
0        PB   bad
1        RS   bad
2        HB   bad
3        PB  at25
4        RS  at25
5        HB  at25


In [59]:
# Step 2: Calculate the estimated marginal means for each factor combination

from patsy import dmatrix

mat=dmatrix("C(algorithm,Treatment(1))*C(prior,Treatment(1))",grid,return_type="matrix")
print(mat)
betas=md.fe_params
print(betas)

emmeans=grid
emmeans["means"]=mat@betas
print(emmeans)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 1.]
 [1. 1. 0. 1. 1. 0.]]
Intercept             0.446291
C(algorithm)[T.PB]    0.293340
C(algorithm)[T.RS]   -0.100037
C(prior)[T.bad]       0.423340
dtype: float64


Exception: Dot product shape mismatch, (4,) vs (6, 6)